# Setup Generale

In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import os
import seaborn as sns

In [2]:
import warnings
warnings.filterwarnings('ignore')

# Funzioni

## Funzioni per il test di significatività

In [3]:
from scipy.stats import mannwhitneyu
from scipy.stats import pearsonr
import statsmodels.stats.proportion as ssp

     test di Mann-Whitney u
     
Il test di Mann-Whitney U è un test non parametrico utilizzato per confrontare due campioni indipendenti e stabilire se esiste una differenza significativa tra di essi. In particolare, il test viene utilizzato per verificare se le mediane dei due campioni sono significativamente diverse.

In [4]:
# group1 e group2 sono due liste di valori
# ritorna p-value => se p-value < 0.05 allora la mediana è significativamente diversa

def test_mann_whitney_u(group1,group2):
    stat, p = mannwhitneyu(group1, group2)
    return p

In [5]:
def calcola_test_mann(df, col, quartili,solo_significativi = False):
    for quartile in quartili:
        
        # valori del quartile
        count = df[df["quartile"] == quartile][col]
        
        # nome del quartile
        group1 = df[df["quartile"] == quartile]["quartile"].unique()
               
        if quartile == "correct_true" or quartile == "wrong_true":
            sovra = "true"
        else:
            sovra = "pants-on-fire"
        
        # nomi quartili sovrapposti
        group2 = df[(df["sovra"] == sovra) & (df["quartile"] != quartile)]["quartile"].unique()
        
        # valori quartili sovrapposti
        nobs = df[(df["sovra"] == sovra) & (df["quartile"] != quartile)][col]
        
        if solo_significativi:
            if test_mann_whitney_u(count,nobs) < 0.05:
                print(group1,group2,"--> pvalue:", round(test_mann_whitney_u(count,nobs),3))
        else:
            print(group1,group2,"--> pvalue: ", test_mann_whitney_u(count,nobs))

    prop test => test di differenza tra due porzioni
Valuta se delle proporzioni sono uguali (p1 = p2 = p3 = .. pk) => ipotesi nulla o se esite una proporzioni significativamente diversa salle altre (ipotesi alternativa) 

In [6]:
# si applica a una specifica dimensione e si passa in input le frequenze assolute di ogni valore della dimensione

# count = particolare (wt/wf/mt/mf)
# nobs = totale di riferimento (true / false)
# ritorna p-value => se p-value < 0.05 allora c'è una proporzione significativamente diversa

def prop_test(count,nobs):
    (chi2, p, arr) = ssp.proportions_chisquare(count,nobs)
    return p

In [7]:
def prop_test_all(count,nobs):
    test_results  = ssp.proportions_chisquare_allpairs(count, nobs, multitest_method='b')
    return test_results 

    p trend test => test 
permette di verificare la presenza di una tendenza lineare tra le proporzioni(l’ipotesi nulla è che non vi sia alcuna tendenza tra di esse; l’alternativa è che vi sia un aumento/decremento lineare della proporzione man mano che si sale/scende nelle categorie. Questo test è da eseguire solo se la variabile categoriale considerata è ordinale.)

In [8]:
# tred test

def tred_test(count,nobs):
    corr, pval = corr, pval = pearsonr(count,nobs)
    return pval

## Funzioni per il plot dei grafici

In [9]:
def set_plot_style():
    # imposto dimensione delle figure e stile
    custom_params = {"axes.spines.right": False, "axes.spines.top": False,'figure.figsize':(4.7,3.27)}
    sns.set_theme(style="ticks", rc=custom_params)

    funzione per generare strip plot (solo puntini) con:
- y = mean della correttezza
- x = quartili
- colori = valori di una dimensione (considerata)

In [10]:
# df_sovra = True o False
# dimensione = dimensione considerata

def plot_strip_plot(df, dimensione, x_value, x_filter = None, y_value = "mean", valori_dimensione = None, order = None):
   
    if order == None:
       order = list(df[x_value].unique())
    
    if valori_dimensione is not None:
        
        tutti = df[dimensione].unique()
        da_tenere = []
        for val in tutti:
            if val not in valori_dimensione:
                da_tenere.append(val) 
                
        for val in da_tenere:
            df = df.loc[df[dimensione] != val]
    
    
    if x_filter is not None:
        diff = order.copy()
        for val in x_filter:
            
            diff.remove(val)
        for val in diff:
            df = df.loc[df[x_value] != val]
            order.remove(val)
    
    ax = sns.stripplot(y=y_value, x=x_value, data=df, hue=dimensione ,jitter=0.1, size=8, order=order, palette="Set2")
    # show the graph
    plt.title(dimensione+" "+x_value, loc="center")
    plt.legend(bbox_to_anchor=(1.02, 1), loc='upper left', borderaxespad=0)
    ax.tick_params(axis='x', rotation=60)
    plt.show()

    funzione per generare un boxplot + stripplot di una dimensione di interesse:
- y = dimensione di interesse
- x = quartitli (wt,wf,ct,wt)

In [11]:
def plot_boxplot(df, dimensione):

    df_copy = df.copy()
    # convert "dimensione" from String to int
    df_copy = df_copy.astype({dimensione:'float'})
    ax = sns.boxplot(palette = "deep",x="quartile", y=dimensione, data=df_copy,order=['correct_false','wrong_false', 'wrong_true','correct_true'])
    ax = sns.stripplot(palette = ["black"],x="quartile", y=dimensione, data=df_copy, jitter=0.1, size=8, order=['correct_false','wrong_false','wrong_true','correct_true'])

    # add title
    plt.title(dimensione, loc="left")
    # show the graph
    plt.legend(bbox_to_anchor=(1.02, 1), loc='upper left', borderaxespad=0)
    plt.show()

    funzione per creare barplot di confronto

In [12]:
from matplotlib.colors import to_rgba

def crea_barplot_all(raccoglitore_dimensioni,tabella_p_value,dimensione,valore,rotation = None,inverted_axes = False):

    fig, ax = plt.subplots(nrows=2, ncols=2, figsize=(15,10))

    if valore == "label":
        fig.tight_layout(h_pad=10, w_pad=5)
    else:
        fig.tight_layout(pad=8.5)
    
    for quartile in quartili:
        
        sub = raccoglitore_dimensioni[dimensione][quartile].sort_values("valore", ascending=False)
        sub_freq_rel = []
        for index, row in sub.iterrows():
            for col in ["freq_rel_quartile", "freq_rel_sovra", "freq_rel_all"]:
                #new_row = [row["valore"],col, sub.at[index,col]]
                new_row = [row[valore],col, sub.at[index,col]]
                sub_freq_rel.append(new_row)

        sub_freq_rel = pd.DataFrame(sub_freq_rel, columns=["valore", "colonna", "frequenza"])
        sub_freq_rel = sub_freq_rel.sort_values(by=["valore"], ascending=True)
        
        if quartile == "wrong_false":
            i = 0
            j = 0
        elif quartile == "wrong_true":
            i = 0
            j = 1
        elif quartile == "correct_false":
            i = 1
            j = 0
        elif quartile == "correct_true":
            i = 1
            j = 1
        
        
        ax[i,j].grid(alpha=0.4)
            
        # colori
        color_dict = {'freq_rel_quartile': to_rgba('salmon', 1),
                    'freq_rel_sovra': to_rgba('paleturquoise', 0.2),
                    'freq_rel_all': to_rgba('mistyrose', 0.2)}   

        if inverted_axes:
            # orrizzontale  
            sns.barplot(data=sub_freq_rel, y = "valore", x = "frequenza", hue ="colonna", ax=ax[i,j], palette=color_dict, orient="h")   
        else:
            #verticale
            #sns.barplot(data=sub_freq_rel, y = "frequenza", x = "valore", hue ="colonna", ax=ax[i,j], palette=["g","indianred", "goldenrod"]) 
            sns.barplot(data=sub_freq_rel, y = "frequenza", x = "valore", hue ="colonna", ax=ax[i,j], palette=color_dict)
            
        
        '''
        aplha_value = 0.35
        
        for bar, alpha in zip(ax[i,j].containers[2], list(aplha_value*np.ones(len(sub_freq_rel["valore"])))):
            bar.set_alpha(alpha)
        for bar, alpha in zip(ax[i,j].containers[0], list(aplha_value*np.ones(len(sub_freq_rel["valore"])))):
            bar.set_alpha(alpha)
        '''
        
        if rotation is not None:
            ax[i,j].tick_params(axis='x', rotation=rotation)
        else:
            ax[i,j].tick_params(axis='x', rotation=30)
        
        if tabella_p_value[dimensione][quartile]["pvalue_quartile_sovra"][0] < 0.05:
            if tabella_p_value[dimensione][quartile]["p_trend"][0] != "NULL":
                ax[i,j].text(0.5, 1.08, "p-value: " + str(round(tabella_p_value[dimensione][quartile]["pvalue_quartile_sovra"][0],3)) + " -- p-trend:" + str(round(tabella_p_value[dimensione][quartile]["p_trend"][0],3))
                        , transform=ax[i,j].transAxes, fontsize=10, fontweight=0, color='red', va='top', ha='center')
                ax[i,j].set_title(dimensione + " -- " + quartile,loc="center", fontsize=15, fontweight=0, color='red', y=1.09)
            else:
                ax[i,j].text(0.5, 1.08, "p-value: " + str(round(tabella_p_value[dimensione][quartile]["pvalue_quartile_sovra"][0],3)) + " -- p-trend:" + tabella_p_value[dimensione][quartile]["p_trend"][0]
                        , transform=ax[i,j].transAxes, fontsize=10, fontweight=0, color='red', va='top', ha='center')
                ax[i,j].set_title(dimensione + " -- " + quartile,loc="center", fontsize=15, fontweight=0, color='red', y=1.09)
            
        else:
            if tabella_p_value[dimensione][quartile]["p_trend"][0] != "NULL":
                ax[i,j].text(0.5, 1.08, "p-value: " + str(round(tabella_p_value[dimensione][quartile]["pvalue_quartile_sovra"][0],3)) + " -- p-trend:" + str(round(tabella_p_value[dimensione][quartile]["p_trend"][0],3))
                        , transform=ax[i,j].transAxes, fontsize=10, fontweight=0, color='black', va='top', ha='center')
                ax[i,j].set_title(dimensione + " -- " + quartile,loc="center", fontsize=15, fontweight=0, color='black', y=1.09)
            else:
                ax[i,j].text(0.5, 1.08, "p-value: " + str(round(tabella_p_value[dimensione][quartile]["pvalue_quartile_sovra"][0],3)) + " -- p-trend:" + tabella_p_value[dimensione][quartile]["p_trend"][0]
                        , transform=ax[i,j].transAxes, fontsize=10, fontweight=0, color='black', va='top', ha='center')
                ax[i,j].set_title(dimensione + " -- " + quartile,loc="center", fontsize=15, fontweight=0, color='black', y=1.09)
            
        
    
    # sistemo legende
    for i in range(2):
        for j in range(2):
            if i == 1 and j == 1:
                ax[i,j].legend(bbox_to_anchor=(1.1, 0.35), loc='upper left', borderaxespad=0)
            else:
                ax[i,j].get_legend().remove()
        
    
   
    plt.show()

In [ ]:
from matplotlib.colors import to_rgba

def crea_barplot_all_rapporto(raccoglitore_dimensioni,tabella_p_value,dimensione,valore,rotation = None,inverted_axes = False):

    fig, ax = plt.subplots(nrows=2, ncols=2, figsize=(15,10))

    if valore == "label":
        fig.tight_layout(h_pad=10, w_pad=5)
    else:
        fig.tight_layout(pad=8.5)
    
    for quartile in quartili:
        
        sub = raccoglitore_dimensioni[dimensione][quartile].sort_values("valore", ascending=False)
        sub_freq_rel = []
        for index, row in sub.iterrows():
            for col in ["freq_rel_quartile", "freq_rel_sovra", "freq_rel_all"]:
                #new_row = [row["valore"],col, sub.at[index,col]]
                new_row = [row[valore],col, sub.at[index,col]]
                sub_freq_rel.append(new_row)

        sub_freq_rel = pd.DataFrame(sub_freq_rel, columns=["valore", "colonna", "frequenza"])
        sub_freq_rel = sub_freq_rel.sort_values(by=["valore"], ascending=True)
        
        if quartile == "wrong_false":
            i = 0
            j = 0
        elif quartile == "wrong_true":
            i = 0
            j = 1
        elif quartile == "correct_false":
            i = 1
            j = 0
        elif quartile == "correct_true":
            i = 1
            j = 1
        
        
        ax[i,j].grid(alpha=0.4)
            
        # colori
        color_dict = {'freq_rel_quartile': to_rgba('salmon', 1),
                    'freq_rel_sovra': to_rgba('paleturquoise', 0.2),
                    'freq_rel_all': to_rgba('mistyrose', 0.2)}   

        if inverted_axes:
            # orrizzontale  
            sns.barplot(data=sub_freq_rel, y = "valore", x = "frequenza", hue ="colonna", ax=ax[i,j], palette=color_dict, orient="h")   
        else:
            #verticale
            #sns.barplot(data=sub_freq_rel, y = "frequenza", x = "valore", hue ="colonna", ax=ax[i,j], palette=["g","indianred", "goldenrod"]) 
            sns.barplot(data=sub_freq_rel, y = "frequenza", x = "valore", hue ="colonna", ax=ax[i,j], palette=color_dict)
            
        
        '''
        aplha_value = 0.35
        
        for bar, alpha in zip(ax[i,j].containers[2], list(aplha_value*np.ones(len(sub_freq_rel["valore"])))):
            bar.set_alpha(alpha)
        for bar, alpha in zip(ax[i,j].containers[0], list(aplha_value*np.ones(len(sub_freq_rel["valore"])))):
            bar.set_alpha(alpha)
        '''
        
        if rotation is not None:
            ax[i,j].tick_params(axis='x', rotation=rotation)
        else:
            ax[i,j].tick_params(axis='x', rotation=30)
        
        if tabella_p_value[dimensione][quartile]["pvalue_quartile_sovra"][0] < 0.05:
            if tabella_p_value[dimensione][quartile]["p_trend"][0] != "NULL":
                ax[i,j].text(0.5, 1.08, "p-value: " + str(round(tabella_p_value[dimensione][quartile]["pvalue_quartile_sovra"][0],3)) + " -- p-trend:" + str(round(tabella_p_value[dimensione][quartile]["p_trend"][0],3))
                        , transform=ax[i,j].transAxes, fontsize=10, fontweight=0, color='red', va='top', ha='center')
                ax[i,j].set_title(dimensione + " -- " + quartile,loc="center", fontsize=15, fontweight=0, color='red', y=1.09)
            else:
                ax[i,j].text(0.5, 1.08, "p-value: " + str(round(tabella_p_value[dimensione][quartile]["pvalue_quartile_sovra"][0],3)) + " -- p-trend:" + tabella_p_value[dimensione][quartile]["p_trend"][0]
                        , transform=ax[i,j].transAxes, fontsize=10, fontweight=0, color='red', va='top', ha='center')
                ax[i,j].set_title(dimensione + " -- " + quartile,loc="center", fontsize=15, fontweight=0, color='red', y=1.09)
            
        else:
            if tabella_p_value[dimensione][quartile]["p_trend"][0] != "NULL":
                ax[i,j].text(0.5, 1.08, "p-value: " + str(round(tabella_p_value[dimensione][quartile]["pvalue_quartile_sovra"][0],3)) + " -- p-trend:" + str(round(tabella_p_value[dimensione][quartile]["p_trend"][0],3))
                        , transform=ax[i,j].transAxes, fontsize=10, fontweight=0, color='black', va='top', ha='center')
                ax[i,j].set_title(dimensione + " -- " + quartile,loc="center", fontsize=15, fontweight=0, color='black', y=1.09)
            else:
                ax[i,j].text(0.5, 1.08, "p-value: " + str(round(tabella_p_value[dimensione][quartile]["pvalue_quartile_sovra"][0],3)) + " -- p-trend:" + tabella_p_value[dimensione][quartile]["p_trend"][0]
                        , transform=ax[i,j].transAxes, fontsize=10, fontweight=0, color='black', va='top', ha='center')
                ax[i,j].set_title(dimensione + " -- " + quartile,loc="center", fontsize=15, fontweight=0, color='black', y=1.09)
            
        
    
    # sistemo legende
    for i in range(2):
        for j in range(2):
            if i == 1 and j == 1:
                ax[i,j].legend(bbox_to_anchor=(1.1, 0.35), loc='upper left', borderaxespad=0)
            else:
                ax[i,j].get_legend().remove()
        
    
   
    plt.show()

In [13]:
from matplotlib.colors import to_rgba

def crea_barplot_quartili(raccoglitore_dimensioni,dimensione,valore):

    
    fig, ax = plt.subplots(nrows=2, ncols=2, figsize=(15,10))

    fig.tight_layout(pad=5.0)
    
    for quartile in quartili:
        
        sub = raccoglitore_dimensioni[dimensione][quartile].sort_values("valore", ascending=False)
        sub_freq_rel = []
        for index, row in sub.iterrows():
            for col in ["freq_rel_quartile"]:
                new_row = [row[valore],col, sub.at[index,col]]
                sub_freq_rel.append(new_row)

        sub_freq_rel = pd.DataFrame(sub_freq_rel, columns=["valore", "colonna", "frequenza"])
        sub_freq_rel = sub_freq_rel.sort_values(by=["valore"], ascending=True)
        
        if quartile == "wrong_false":
            i = 0
            j = 0
        elif quartile == "wrong_true":
            i = 0
            j = 1
        elif quartile == "correct_false":
            i = 1
            j = 0
        elif quartile == "correct_true":
            i = 1
            j = 1
        
        
        sns.barplot(data=sub_freq_rel,y = "valore", x = "frequenza", hue ="colonna", ax=ax[i,j],palette="rocket")   
                
        ax[i,j].set_title(dimensione + " " + quartile,loc="center", fontsize=15, fontweight=0, color='black', y=1.01)
        ax[i,j].set_subtitle("p-prop:" + raccoglitore_dimensioni[dimensione][quartile]["valore_pvalue"] + " , " + "p-trend:"+ raccoglitore_dimensioni[dimensione][quartile]["p_trend"], loc="center", fontsize=10, fontweight=0, color='black', y=1.01)
    
    # sistemo legende
    for i in range(2):
        for j in range(2):
            if i == 1 and j == 1:
                ax[i,j].legend(bbox_to_anchor=(1.1, 0.35), loc='upper left', borderaxespad=0)
            else:
                ax[i,j].get_legend().remove()
        
    
   
    plt.show()


In [14]:
from matplotlib.colors import to_rgba

def crea_barplot_gruppi_quartili(raccoglitore_dimensioni,dimensione,valore,gruppo):

    fig, ax = plt.subplots(nrows=2, ncols=2, figsize=(15,10))

    fig.tight_layout(pad=10.0)
    
    for quartile in quartili:
        
        sub = raccoglitore_dimensioni[dimensione][quartile].sort_values("valore", ascending=False)
        sub_freq_rel = []
        for index, row in sub.iterrows():
            for col in ["freq_rel_quartile"]:
                new_row = [row[valore],col, sub.at[index,col]]
                sub_freq_rel.append(new_row)

        sub_freq_rel = pd.DataFrame(sub_freq_rel, columns=["valore", "colonna", "frequenza"])
        sub_freq_rel = sub_freq_rel.sort_values(by=["valore"], ascending=True)
        
        if quartile == "wrong_false":
            i = 0
            j = 0
        elif quartile == "wrong_true":
            i = 0
            j = 1
        elif quartile == "correct_false":
            i = 1
            j = 0
        elif quartile == "correct_true":
            i = 1
            j = 1
        
        
        sns.barplot(data=sub_freq_rel,y = "valore", x = "frequenza", hue ="colonna", ax=ax[i,j])   
        sns.color_palette("crest", as_cmap=True)
        
        
        ax[i,j].set_title(dimensione + " " + quartile,loc="center", fontsize=15, fontweight=0, color='black', y=1.01)
    
    # sistemo legende
    for i in range(2):
        for j in range(2):
            if i == 1 and j == 1:
                ax[i,j].legend(bbox_to_anchor=(1.1, 0.35), loc='upper left', borderaxespad=0)
            else:
                ax[i,j].get_legend().remove()
        
    
   
    plt.show()

     funzione per creare box plot + stripplot

In [15]:
def crea_boxplot_e_stripplot(df, x_dimensione, y_dimensione, order = ['correct_false','other_false','wrong_false', 'wrong_true','other_true','correct_true'], gruppi = None, labels = None,titolo = None):

    if gruppi == None:
        ax = sns.boxplot(palette = "pastel",x=x_dimensione, y=y_dimensione, data=df,order = order)
        
        ax = sns.stripplot(palette = "dark",x=x_dimensione, y=y_dimensione, data=df, jitter=0.2, size=5, order = order, alpha=0.95)

        # add title
        plt.title(x_dimensione + " -- vs -- " + y_dimensione , loc="center", fontsize=15, fontweight=0, color='black', y=1.01)
        plt.grid(alpha=0.3)
        ax.tick_params(axis='x', rotation=90)
        # show the graph
        plt.show()
    else:
        new_df = []
        i = 0
        for gruppo in gruppi:
            for subgroup in gruppo:
                arr = df[df[x_dimensione] == subgroup][y_dimensione]
                for val in arr:
                    new_df.append([labels[i],val])
            i += 1
        new_df = pd.DataFrame(new_df, columns=["gruppo",y_dimensione])
        
        ax = sns.boxplot(palette = "pastel",x="gruppo", y=y_dimensione, data=new_df)
        
        ax = sns.stripplot(palette = "dark",x="gruppo", y=y_dimensione, data=new_df, jitter=0.2, size=5, alpha=0.95)

        # add title
        plt.title(titolo + " -- vs -- " + y_dimensione , loc="center", fontsize=15, fontweight=0, color='black', y=1.01)
        plt.grid(alpha=0.3)
        ax.tick_params(axis='x', rotation=90)
        # show the graph
        plt.show()
        

    funzione per regressione lineare

In [16]:
from scipy.stats import pearsonr
from sklearn.metrics import r2_score

def plot_regressione_lineare_RQ(df,dimensioni_da_analizzare,lunghezza_testo):
    
        for dimensione in dimensioni_da_analizzare:
                #print(dimensione)
                fig, ax = plt.subplots(1,2, figsize=(12.02,3),sharey=True)
                        
                for sovra in ["pants-on-fire","true"]:
                    
                        if sovra == "true":
                                j = 1
                        else:
                                j = 0
                                
                        sub = df.copy()
                                        
                        sub = sub[sub["doc_fact_check_ground_truth_label"] == sovra]
                        
                        test = { "correct_true" : "#FFA630",
                                "other_true": "#95C247",
                                "wrong_true": "#4DA1A9",
                                "correct_false" : "#2E5077",
                                "other_false": "#611C35",
                                "wrong_false": "#A68DCC"
                                
                        }
                    
                        sns.regplot(x="mean", 
                                        y=sub[dimensione[:-lunghezza_testo]+"_mean"], 
                                        data=sub,
                                        ax=ax[j],
                                        scatter_kws={'s':0},
                                        color="r"
                                        )
                        
                        sns.scatterplot(x="mean", 
                                        y=sub[dimensione[:-lunghezza_testo]+"_mean"], 
                                        hue = "quartile",
                                        palette=test,
                                        data=sub,
                                        ax=ax[j]
                                        )
                    
                        # calcolo correlazione
                        sub[dimensione] = sub[dimensione].astype("float")
                        
                        sub.drop_duplicates(subset = ["doc_id"], inplace = True)
                        sub[dimensione] = sub[dimensione].astype("float")
                        
                        tabella = pearsonr(sub['mean'], sub[dimensione])
                        
                        coef_corr = tabella[0]
                        pval = tabella[1]
                        r2 = coef_corr**2
                    
                        ax[j].grid(alpha=0.3)
                        
                        # legenda
                        
                        '''
                        ax[j].plot([0,1],[0,1], transform=ax[j].transAxes, color="red", alpha=0, 
                                        label="R^2 = " + str(round(r2,4)))
                        ax[j].plot([0,1],[0,1], transform=ax[j].transAxes, color="red", alpha=0, 
                                        label="p = " + str(round(pval,4)))
                        
                        '''
                        
                        if pval < 0.05 :
                                ax[j].text(0.5, 1.2, "r2 = " + str(round(r2,4)) + " -- p = " + str(round(pval,4)),
                                        transform=ax[j].transAxes, fontsize=10, fontweight=0, color='red', va='top', ha='center')
                                #ax[j].set_title(str(dimensione[:-6]) + " -- " + sovra,loc="center", fontsize=15, fontweight=0, color='red', y=1.23)
                                ax[j].set_title(sovra,loc="center", fontsize=15, fontweight=0, color='red', y=1.23)
                        
                        else :
                                ax[j].text(0.5, 1.2, "r2 = " + str(round(r2,4)) + " -- p = " + str(round(pval,4)),
                                        transform=ax[j].transAxes, fontsize=10, fontweight=0, color='black', va='top', ha='center')
                                #ax[j].set_title(str(dimensione[:-6]) + " -- " + sovra,loc="center", fontsize=15, fontweight=0, color='black', y=1.23)
                                ax[j].set_title(sovra,loc="center", fontsize=15, fontweight=0, color='black', y=1.23)
                        
                        ax[j].legend(bbox_to_anchor=(1, -0.27), borderaxespad=0, ncol = 3)
                        
                print("\centering\n\\begin{subfigure}{.85\linewidth}\n\centering")    
                print("\includegraphics[width=\linewidth]{tesi/media/new/regressione_"+dimensione+".png}\n\caption{Analisi di regressione lineare della dimensione " + dimensione[:-6]+"}\n\label{cap:regressione_lineare_" + dimensione[:-6] + "}")
                print("\end{subfigure}")
                        
                #plt.show()
                plt.savefig("plots/regressione_"+dimensione+".png", dpi=500, bbox_inches='tight')  

In [17]:
'''
from matplotlib.legend import Legend
def plot_regressione_lineare(df,dimensioni_da_analizzare):
    fig, ax = plt.subplots(len(dimensioni_da_analizzare),2, figsize=(15,45))

    fig.tight_layout(h_pad=14, w_pad=2.5)

    i = -1

    for dimensione in dimensioni_da_analizzare:
            print(dimensione)
            i += 1        
            for sovra in ["pants-on-fire","true"]:
                    
                    if sovra == "true":
                            j = 1
                    else:
                            j = 0
                            
                    sub = df.copy()
                                    
                    sub = sub[sub["doc_fact_check_ground_truth_label"] == sovra]
                    
                    test = { "correct_true" : "#FFA630",
                            "other_true": "#95C247",
                            "wrong_true": "#4DA1A9",
                            "correct_false" : "#2E5077",
                            "other_false": "#611C35",
                            "wrong_false": "#A68DCC"
                            
                    }
                    
                    sns.regplot(x="mean", 
                                   y=np.asarray(sub[dimensione],dtype = np.float64), 
                                    data=sub,
                                    ax=ax[i,j],
                                    scatter_kws={'s':0},
                                    color="r"
                                    )
                    
                    sns.scatterplot(x="mean", 
                                    y=np.asarray(sub[dimensione],dtype = np.float64),  
                                    hue = "quartile",
                                    palette=test,
                                    data=sub,
                                    ax=ax[i,j]
                                    )
                    
                    # calcolo correlazione
                    sub[dimensione] = sub[dimensione].astype("float")
                    
                    sub.drop_duplicates(subset = ["doc_id"], inplace = True)
                    sub[dimensione] = sub[dimensione].astype("float")
                    
                    tabella = pearsonr(sub['mean'], sub[dimensione])
                    
                    r2 = tabella[0]
                    pval = tabella[1]
                    
                    ax[i,j].grid(alpha=0.3)
                    # legenda
                    ax[i,j].plot([0,1],[0,1], transform=ax[i,j].transAxes, color="red", alpha=0, 
                                 label="R^2 = " + str(round(r2,4)))
                    ax[i,j].plot([0,1],[0,1], transform=ax[i,j].transAxes, color="red", alpha=0, 
                                 label="p = " + str(round(pval,4)))
                    if j == 0:
                        ax[i,j].legend(bbox_to_anchor=(0.93, -0.12), borderaxespad=0, ncol = 4)
                    else:
                        ax[i,j].legend(bbox_to_anchor=(0.93, -0.12), borderaxespad=0, ncol = 4)
    plt.show()
'''

'\nfrom matplotlib.legend import Legend\ndef plot_regressione_lineare(df,dimensioni_da_analizzare):\n    fig, ax = plt.subplots(len(dimensioni_da_analizzare),2, figsize=(15,45))\n\n    fig.tight_layout(h_pad=14, w_pad=2.5)\n\n    i = -1\n\n    for dimensione in dimensioni_da_analizzare:\n            print(dimensione)\n            i += 1        \n            for sovra in ["pants-on-fire","true"]:\n                    \n                    if sovra == "true":\n                            j = 1\n                    else:\n                            j = 0\n                            \n                    sub = df.copy()\n                                    \n                    sub = sub[sub["doc_fact_check_ground_truth_label"] == sovra]\n                    \n                    test = { "correct_true" : "#FFA630",\n                            "other_true": "#95C247",\n                            "wrong_true": "#4DA1A9",\n                            "correct_false" : "#2E5077",\n       

In [18]:
from matplotlib.legend import Legend
def plot_regressione_lineare(df,dimensioni_da_analizzare):
    

    for dimensione in dimensioni_da_analizzare:
        print(dimensione)
        fig, ax = plt.subplots(1,2, figsize=(12.02,3),sharey=True)        
        for sovra in ["pants-on-fire","true"]:
                
                if sovra == "true":
                        j = 1
                else:
                        j = 0
                        
                sub = df.copy()
                                
                sub = sub[sub["doc_fact_check_ground_truth_label"] == sovra]
                
                test = { "correct_true" : "#FFA630",
                        "other_true": "#95C247",
                        "wrong_true": "#4DA1A9",
                        "correct_false" : "#2E5077",
                        "other_false": "#611C35",
                        "wrong_false": "#A68DCC"
                        
                }
                    
                sns.regplot(x="mean", 
                                y=np.asarray(sub[dimensione],dtype = np.float64), 
                                data=sub,
                                ax=ax[j],
                                scatter_kws={'s':0},
                                color="r"
                                )
                
                sns.scatterplot(x="mean", 
                                y=np.asarray(sub[dimensione],dtype = np.float64),  
                                hue = "quartile",
                                palette=test,
                                data=sub,
                                ax=ax[j]
                                )
                    
                # calcolo correlazione
                sub[dimensione] = sub[dimensione].astype("float")
                
                sub.drop_duplicates(subset = ["doc_id"], inplace = True)
                sub[dimensione] = sub[dimensione].astype("float")
                
                tabella = pearsonr(sub['mean'], sub[dimensione])
                    
                r2 = tabella[0]
                pval = tabella[1]
                
                ax[j].grid(alpha=0.3)
                    
                if pval < 0.05 :
                        ax[j].text(0.5, 1.2, "R^2 = " + str(round(r2,4)) + " -- p = " + str(round(pval,4)),
                                transform=ax[j].transAxes, fontsize=10, fontweight=0, color='red', va='top', ha='center')
                        ax[j].set_title(str(dimensione) + " -- " + sovra,loc="center", fontsize=15, fontweight=0, color='red', y=1.23)
                
                else :
                        ax[j].text(0.5, 1.2, "R^2 = " + str(round(r2,4)) + " -- p = " + str(round(pval,4)),
                                transform=ax[j].transAxes, fontsize=10, fontweight=0, color='black', va='top', ha='center')
                        ax[j].set_title(str(dimensione) + " -- " + sovra,loc="center", fontsize=15, fontweight=0, color='black', y=1.23)
                ax[j].legend(bbox_to_anchor=(1, -0.27), borderaxespad=0, ncol = 3)
        plt.show()

## Funzioni di Supporto

    Funzione per determinare la frequenza assoluta
- data una dimensione da analizzare
- dato il valore di interesse

calcola la frequenza assoluta del valore rispetto al totale --> numero di occorrenze di un valore nel df

In [19]:
def calcola_frequenza_assoluta(df, dimensione, valore_dimensione, sovra_categoria):
        if sovra_categoria == 'all':
                #sub = df.drop_duplicates(subset=['worker_id'])
                sub = df
                return str(list(sub[dimensione])).count(str(valore_dimensione))
        else:
                sub = df[df['quartile'] == sovra_categoria]
                #sub = sub.drop_duplicates(subset=['worker_id'])
                return str(list(sub[dimensione])).count(str(valore_dimensione))
        

    Funzione per aggiungere un valore a una determinata colonna di una determinata riga di un dataframe 

In [20]:
# scorro df ipotizzando che il valore di doc_id sia unico

def aggiungi_valore(df,colonna,valore,doc_id):
    df.loc[df['doc_id'] == doc_id, colonna] = valore
    return df

    Ordina tabella secondo una dimensione

In [21]:
def ordina_tabella(df,dimensione,ordine):
    df = df.sort_values(by=[dimensione], ascending=ordine)
    return df

    voglio una tabella che mi dica per ogni valore considerato di una dimensione:
- frequenza assoluta del valore nella sotto categoria
- frequenza assoluta del valore nella sovra categoria
- frequenza assoluta del valore in generale (tutto)

df => df_all

In [22]:
def crea_subDF_con_frequenze(df, sovra_categoria, quartile, dimensione,mapping = None):
    res_assolute = []
    res_relative = []
    for valore_dimensione in df[dimensione].unique():
        if mapping is not None:
            label = mapping[dimensione[:-10]][valore_dimensione]
        freq_ass_all = calcola_frequenza_assoluta(df, dimensione, valore_dimensione, "all")
        freq_ass_sovra = calcola_frequenza_assoluta(df[df["doc_fact_check_ground_truth_label"] == sovra_categoria], dimensione, valore_dimensione, "all")
        freq_ass_quartile = calcola_frequenza_assoluta(df, dimensione, valore_dimensione, quartile)
        if mapping is not None:
            res_assolute.append([valore_dimensione,label,freq_ass_all,freq_ass_sovra,freq_ass_quartile]) 
        else:
            res_assolute.append([valore_dimensione,freq_ass_all,freq_ass_sovra,freq_ass_quartile])
    if mapping is not None:
        res_assolute = pd.DataFrame(res_assolute, columns=["valore","label", "freq_ass_all", "freq_ass_sovra", "freq_ass_quartile"])     
    else:
        res_assolute = pd.DataFrame(res_assolute, columns=["valore", "freq_ass_all", "freq_ass_sovra", "freq_ass_quartile"])
        
    res_relative = []
    
    if mapping is not None:
        start = 2
    else:
        start = 1
    
    for col in res_assolute.columns[start:]:
        
        tot = res_assolute[col].sum()
        colonna = []
        for val in res_assolute[col]:
            colonna.append(val/tot)    
        res_relative.append(colonna)
            
    res_relative = pd.DataFrame(res_relative).T
    res_relative.columns = ["freq_rel_all", "freq_rel_sovra", "freq_rel_quartile"]

    sub = pd.concat([res_assolute, res_relative], axis=1, join='inner')
    return ordina_tabella(sub,"valore",True)

    aggiungi roba al df all

In [23]:
# unisci due df tramite colonna_uguale

def aggiungi_colonna(df_totale,df_secondario, colonna,colonna_uguale_primo, colonna_uguale_secondario):
    df_totale[str(colonna)] = 1

    for index, row in df_totale.iterrows():
        id = row[str(colonna_uguale_primo)]
        df_totale.at[index,str(colonna)] = df_secondario[df_secondario[str(colonna_uguale_secondario)] == id][str(colonna)].values[0]

    raccoglitore dimensioni

In [24]:
def crea_raccoglitore_dimensioni(df,dimensioni_da_analizzare,quartili,mapping=None):
    # definisco un dizionario che conterrà i df delle dimensioni
    raccoglitore_dimensioni = {}

    # inizializzo il dizionario
    for dimensione in dimensioni_da_analizzare:
        raccoglitore_dimensioni[dimensione] = {}

    for dimensione in dimensioni_da_analizzare:
        for quartile in quartili:
            
            # set sovra
            if quartile == "wrong_false" or quartile == "correct_false":
                sovra = "pants-on-fire"
            else:
                sovra = "true"
            
            # creo il df
            if mapping == None:
                raccoglitore_dimensioni[dimensione][quartile] = crea_subDF_con_frequenze(df, sovra, quartile, dimensione)
            else:
                raccoglitore_dimensioni[dimensione][quartile] = crea_subDF_con_frequenze(df, sovra, quartile, dimensione,mapping)
    
    return raccoglitore_dimensioni

    costruisci tabella p-value

In [25]:
def is_float(string):
    try:
        float(string)
        return True
    except ValueError:
        return False

In [26]:
from scipy.stats import linregress

def costruisci_tabella_pvalue(raccoglitore_dimensioni,dimensioni_da_analizzare,quartili,ignora_per_ptrend = [],usa_mapping = False, mapping_per_ptrend = {}):
    # definisco un dizionario che conterrà i df dei p-value
    raccoglitore_pvalue = {}

    # inizializzo il dizionario
    for dimensione in dimensioni_da_analizzare:
        raccoglitore_pvalue[dimensione] = {}

    for dimensione in dimensioni_da_analizzare:
        for quartile in quartili:
            
            # estraggo le frequenze assolute dal dizionario raccoglitore_dimensioni
            
            freq_ass_quartile = raccoglitore_dimensioni[dimensione][quartile]["freq_ass_quartile"]
            freq_ass_all = raccoglitore_dimensioni[dimensione][quartile]["freq_ass_all"]
            freq_ass_sovra = raccoglitore_dimensioni[dimensione][quartile]["freq_ass_sovra"]
            
            # calcolo i p-value
            
            p_value_quartile_all = prop_test(freq_ass_quartile,freq_ass_all)
            p_value_sovra_all = prop_test(freq_ass_sovra,freq_ass_all)
            
            # per calculare il p-value del quartile rispetto alla sovra categoria devo eliminare i valori 0 => poichè ci possono essere valori che non ci sono ne nel quartile ne nella sovra categoria
            
            for i in range(0,len(freq_ass_sovra)):
                if freq_ass_sovra[i] == 0 or freq_ass_sovra[i] == "0":
                    freq_ass_sovra.pop(i)
                    freq_ass_quartile.pop(i)
                
            p_value_quartile_sovra = prop_test(freq_ass_quartile,freq_ass_sovra)
            
            # calcolo p-trend
            if dimensione in ignora_per_ptrend:
                p_trend = "NULL"
            else:
                
                if usa_mapping == False:
                
                    x = raccoglitore_dimensioni[dimensione][quartile]["valore"].astype(float)
                    y = [i for i in raccoglitore_dimensioni[dimensione][quartile][["freq_ass_quartile"]].astype(float)["freq_ass_quartile"]]
                
                    p_trend = linregress(x,y)[3]
                    
                else:
                    x = []
                    for valore in raccoglitore_dimensioni[dimensione][quartile]["valore"]:
                        x.append(mapping_per_ptrend[dimensione][valore])
                    y = [i for i in raccoglitore_dimensioni[dimensione][quartile][["freq_ass_quartile"]].astype(float)["freq_ass_quartile"]]
                
                    p_trend = linregress(x,y)[3]
            
            
            # creo row
            
            row = [[p_value_quartile_all,
                p_value_quartile_sovra,
                p_value_sovra_all,
                p_trend
            ]]
            
            # creo il df
            
            raccoglitore_pvalue[dimensione][quartile] = pd.DataFrame(row, columns=["pvalue_quartile_all", "pvalue_quartile_sovra", "pvalue_sovra_all", "p_trend"])
    
    return raccoglitore_pvalue

    ritorna significativi

In [27]:
def filtra_siglificativi(raccoglitore_pvalue,dimensioni_da_analizzare, quartili, valore_significativo,valore_confronto):
    
    significativi = []

    for dimensione in dimensioni_da_analizzare:
        for quartile in quartili:
            for val in raccoglitore_pvalue[dimensione][quartile]:
                if val != "p_trend" and float(raccoglitore_pvalue[dimensione][quartile][val][0]) < valore_significativo:
                    row = [dimensione, quartile, val, raccoglitore_pvalue[dimensione][quartile][val][0], raccoglitore_pvalue[dimensione][quartile]["p_trend"][0]]
                    significativi.append(row)
    
    significativi = pd.DataFrame(significativi, columns=["dimensione", "quartile", "confronto", "valore_pvalue","p_trend"])
    
    return significativi[significativi["confronto"] == valore_confronto].sort_values(by=["valore_pvalue"], ascending=True)

    calcola scarto e scarto medio 

In [28]:
def calcola_scarto(df):
    df["scarto"] = 0
    for row in df.index:
        df.loc[row,"scarto"] = abs(df.loc[row,"doc_truthfulness_value"] - df.loc[row,"doc_fact_check_ground_truth_value"])

    df["scarto_medio"] = 0
    for w in df["worker_id"].unique():
        df.loc[df["worker_id"] == w, "scarto_medio"] = df[df["worker_id"] == w]["scarto"].mean()
    
    return df

    barplot quartili per gruppi 

In [29]:

from matplotlib.colors import to_rgba
from math import ceil

def crea_barplot_gruppi_quartili(raccoglitore_dimensioni,dimensioni_da_analizzare,gruppo,label,names = None, numero_per_riga = 3):
    c = numero_per_riga
    r = ceil(len(dimensioni_da_analizzare) / c)
    fig, ax = plt.subplots(nrows=r, ncols=c, figsize=(15,10))
    fig.tight_layout(pad=5.0)
    i = 0
    j = 0
    
    if type(gruppo[0]) == list:
        # caso in cui il gruppo è composto da più sotto gruppi
        
        for dimensione in dimensioni_da_analizzare:

            sub_df = []
            a = 0
            for gr in gruppo:
                for sgr in gr:
                    sub = raccoglitore_dimensioni[dimensione][sgr]
                    for index, row in sub.iterrows():
                        if label:
                            sub_df.append([names[a],row["label"],row["valore"],row["freq_rel_quartile"]])
                        else:
                            sub_df.append([names[a],row["valore"],row["freq_rel_quartile"]])
                a = a + 1
            
            if label:        
                sub_df = pd.DataFrame(sub_df, columns=["gruppo","label","valore","freq_rel_quartile"])
            else:
                sub_df = pd.DataFrame(sub_df, columns=["gruppo","valore","freq_rel_quartile"])
            sns.barplot(data=sub_df,y = "freq_rel_quartile",x = "valore", hue ="gruppo", palette = "Set2", ax=ax[i,j])
            ax[i,j].grid(alpha=0.4)
            ax[i,j].set_title(dimensione,loc="center", fontsize=15, fontweight=0, color='black', y=1.01)
            
            j = j + 1
            if j == c:
                j = 0
                i = i + 1
        
    else: 
        # caso in cui il gruppo è composto da un solo sotto gruppo
        for dimensione in dimensioni_da_analizzare:

            sub_df = []

            for gr in gruppo:
                sub = raccoglitore_dimensioni[dimensione][gr]
                for index, row in sub.iterrows():
                    if label:
                        sub_df.append([gr,row["label"],row["valore"],row["freq_rel_quartile"]])
                    else:
                        sub_df.append([gr,row["valore"],row["freq_rel_quartile"]])
                    
            if label:        
                sub_df = pd.DataFrame(sub_df, columns=["gruppo","label","valore","freq_rel_quartile"])
            else:
                sub_df = pd.DataFrame(sub_df, columns=["gruppo","valore","freq_rel_quartile"])

            sns.barplot(data=sub_df,y = "freq_rel_quartile", x = "valore", hue ="gruppo", palette = "Set2", ax=ax[i,j])
            ax[i,j].grid(alpha=0.4)
            ax[i,j].set_title(dimensione,loc="center", fontsize=15, fontweight=0, color='black', y=1.01)
            
            j = j + 1
            if j == c:
                j = 0
                i = i + 1
        
    plt.show()

In [30]:
def calcola_p_value_gruppi_quartili(raccoglitore_dimensioni,dimensioni_da_analizzare,gruppi_quartili,label,name_gruppi, solo_significativi = False):

    raccoglitore_gruppo = {}

    for dimensione in dimensioni_da_analizzare:
        raccoglitore_gruppo[dimensione] = {}
        for name in name_gruppi:
            raccoglitore_gruppo[dimensione][name] = {}
            for val in raccoglitore_dimensioni[dimensione]["correct_false"]["valore"]:
                if label:
                    raccoglitore_gruppo[dimensione][name][val] = {"label":"","freq_ass_quartile":0,"freq_rel_quartile":0,"freq_ass_all":0,"freq_rel_all":0}
                else:
                    raccoglitore_gruppo[dimensione][name][val] = {"freq_ass_quartile":0,"freq_rel_quartile":0,"freq_ass_all":0,"freq_rel_all":0}



    for dimensione in dimensioni_da_analizzare:
        i = 0
        for gruppo in gruppi_quartili:
            name = name_gruppi[i]
            for valore in raccoglitore_gruppo[dimensione][name]:
                
                for quartile in gruppo:
                    for index,row in raccoglitore_dimensioni[dimensione][quartile].iterrows():
                        
                        if row["valore"] == valore :
                            if label:
                                raccoglitore_gruppo[dimensione][name][valore]["label"] = row["label"]
                            raccoglitore_gruppo[dimensione][name][valore]["freq_ass_all"] = row["freq_ass_all"]
                            raccoglitore_gruppo[dimensione][name][valore]["freq_rel_all"] = row["freq_rel_all"]
                            raccoglitore_gruppo[dimensione][name][valore]["freq_ass_quartile"] += row["freq_ass_quartile"]

                            
            raccoglitore_gruppo[dimensione][name] = pd.DataFrame(raccoglitore_gruppo[dimensione][name]).T
            
            somma = raccoglitore_gruppo[dimensione][name]["freq_ass_quartile"].sum()
            for index,row in raccoglitore_gruppo[dimensione][name].iterrows():
                row["freq_rel_quartile"] = row["freq_ass_quartile"] / somma
            
            valore_significativo = 0.05

            if solo_significativi:
                
                if prop_test(raccoglitore_gruppo[dimensione][name]["freq_ass_quartile"],
                                raccoglitore_gruppo[dimensione][name]["freq_ass_all"]) < valore_significativo:
                    print(name,dimensione, prop_test(raccoglitore_gruppo[dimensione][name]["freq_ass_quartile"],
                                raccoglitore_gruppo[dimensione][name]["freq_ass_all"]))
            else:
            
                if prop_test(raccoglitore_gruppo[dimensione][name]["freq_ass_quartile"],
                                raccoglitore_gruppo[dimensione][name]["freq_ass_all"]) < valore_significativo:
                    print("!!! -> ", name,dimensione, prop_test(raccoglitore_gruppo[dimensione][name]["freq_ass_quartile"],
                                raccoglitore_gruppo[dimensione][name]["freq_ass_all"]))
                else:
                    print(name,dimensione, prop_test(raccoglitore_gruppo[dimensione][name]["freq_ass_quartile"],
                                raccoglitore_gruppo[dimensione][name]["freq_ass_all"]))
     
            
            i += 1
          

In [31]:
def crea_barplot(tabella,quartile):
    tab = []
    for index,row in tabella.iterrows():
        r = [index,row["scarto_medio_"+quartile], "scarto_medio_"+quartile]
        tab.append(r)
        #r = [index,row["sovra"], "sovra"]
        #tab.append(r)
        r = [index,row["scarto_medio_all"], "scarto_medio_all"]
        tab.append(r)
    tab = pd.DataFrame(tab, columns = ["index","valore","tipo"])
    tab["valore"] = tab["valore"].astype("float")
    sns.barplot(x="index", y="valore", hue="tipo", data=tab)
    plt.show()

In [32]:
def analizza_scarto(df_for_Q1,quartili,dimensioni_da_analizzare,solo_quartile_vs_sovra = False):
    
    for dimensione in dimensioni_da_analizzare:

        sub_df_quartile = []

        for quartile in quartili:
            
            sub = {}
            count = {}
            
            for index, row in df_for_Q1.iterrows():
                if row[dimensione] not in sub:
                    sub[row[dimensione]] = 0
                    count[row[dimensione]] = 0
            
            for index, row in df_for_Q1[df_for_Q1["quartile"] == quartile].iterrows():
                sub[row[dimensione]] += float(row["scarto"])
                count[row[dimensione]] += 1
            for val in sub:
                if count[val] > 0:
                    sub_df_quartile.append([val,sub[val],quartile,count[val],sub[val]/count[val]])
                else:
                    sub_df_quartile.append([val,sub[val],quartile,count[val],0])
            
        sub_df_quartile = pd.DataFrame(sub_df_quartile, columns=[dimensione,"scarto_totale","quartile","count","scarto_medio"])

        ## sovra
        #false

        sub = {}
        count = {}

        for index, row in df_for_Q1.iterrows():
            if row[dimensione] not in sub:
                sub[row[dimensione]] = 0
                count[row[dimensione]] = 0



        sub_df_sovra = []

        for index, row in df_for_Q1.iterrows():
            if df_for_Q1.loc[index,"quartile"] == "wrong_false" or df_for_Q1.loc[index,"quartile"] == "correct_false" or df_for_Q1.loc[index,"quartile"] == "other_false":
                sub[row[dimensione]] += float(row["scarto"])
                count[row[dimensione]] += 1

        for val in sub:
                if count[val] > 0:
                    sub_df_sovra.append([val,sub[val],"false",count[val],sub[val]/count[val]])
                else:
                    sub_df_sovra.append([val,sub[val],"false",count[val],0])  
        #true   

        sub = {}
        count = {}

        for index, row in df_for_Q1.iterrows():
            if row[dimensione] not in sub:
                sub[row[dimensione]] = 0
                count[row[dimensione]] = 0
                
        for index, row in df_for_Q1.iterrows():
            if df_for_Q1.loc[index,"quartile"] == "wrong_true" or df_for_Q1.loc[index,"quartile"] == "correct_true" or df_for_Q1.loc[index,"quartile"] == "other_true":
                sub[row[dimensione]] += float(row["scarto"])
                count[row[dimensione]] += 1

        for val in sub:
                if count[val] > 0:
                    sub_df_sovra.append([val,sub[val],"true",count[val],sub[val]/count[val]])
                else:
                    sub_df_sovra.append([val,sub[val],"true",count[val],0])  
                    
        sub_df_sovra = pd.DataFrame(sub_df_sovra, columns=[dimensione,"scarto_totale","sovra","count","scarto_medio"])


        # all

        sub = {}
        count = {}

        for index, row in df_for_Q1.iterrows():
            if row[dimensione] not in sub:
                sub[row[dimensione]] = 0
                count[row[dimensione]] = 0

        sub_df_all = []

        for index, row in df_for_Q1.iterrows():
            sub[row[dimensione]] += float(row["scarto"])
            count[row[dimensione]] += 1

        for val in sub:
                if count[val] > 0:
                    sub_df_all.append([val,sub[val],"all",count[val],sub[val]/count[val]])
                else:
                    sub_df_all.append([val,sub[val],"all",count[val],0])  
                    
        sub_df_all = pd.DataFrame(sub_df_all, columns=[dimensione,"scarto_totale","sovra","count","scarto_medio"])

        tabelle = {}
        
        # calcolo p-value e print tabella
        
        for quartile in quartili:
            tabella = []
            row1 = list(sub_df_quartile[sub_df_quartile["quartile"] == quartile]["scarto_medio"])
            if quartile == "wrong_true" or quartile == "correct_true" or quartile == "other_true":
                row2 = list(sub_df_sovra[sub_df_sovra["sovra"] == "true"]["scarto_medio"])
            else:
                row2 = list(sub_df_sovra[sub_df_sovra["sovra"] == "false"]["scarto_medio"])
            row3 = list(sub_df_all["scarto_medio"])
                
            tabella = [row1,row2,row3]

            tabella = pd.DataFrame(tabella, columns = list(sub_df_all[dimensione])).T
            tabella.columns = ["scarto_medio_"+quartile,"scarto_medio_sovra","scarto_medio_all"]
            tabelle[quartile] = tabella
            pvalue = prop_test(tabella["scarto_medio_"+quartile],tabella["scarto_medio_sovra"])
            if pvalue < 0.05:
                print("quartile vs sovra => ",dimensione, quartile,"->", pvalue)
                # crea_barplot(tabelle[quartile],quartile)
            
            if solo_quartile_vs_sovra == False:
                
                pvalue = prop_test(tabella["scarto_medio_"+quartile],tabella["scarto_medio_all"])
                
                if pvalue < 0.05:
                    print("quartile vs all => ",dimensione, quartile,"->", pvalue)



## Anova

In [1]:
import scipy.stats as stats
import scipy.stats as stats

def group_comparison(groups, factors=None):
    num_groups = len(groups)
    
    if num_groups <= 2:
        # Esegue il test t di Student per due gruppi
        group1 = groups[0]
        group2 = groups[1]
        _, p_value = stats.ttest_ind(group1, group2)
        # print("t-student")
        return p_value
    elif factors is not None:
        # Esegue l'ANOVA two-way
        _, p_value = stats.f_oneway(*groups, *factors)
        # print("two-way")
        return p_value
    else:
        # Esegue l'ANOVA one-way
        # print("one-way")
        _, p_value = stats.f_oneway(*groups)
        
        return p_value

In [ ]:
from scipy.stats import linregress

def costruisci_tabella_pvalue_anova(raccoglitore_dimensioni,dimensioni_da_analizzare,quartili,ignora_per_ptrend = [],usa_mapping = False, mapping_per_ptrend = {}):
    # definisco un dizionario che conterrà i df dei p-value
    raccoglitore_pvalue = {}

    # inizializzo il dizionario
    for dimensione in dimensioni_da_analizzare:
        raccoglitore_pvalue[dimensione] = {}

    for dimensione in dimensioni_da_analizzare:
        for quartile in quartili:
            
            
            # estraggo le frequenze assolute dal dizionario raccoglitore_dimensioni
            
            freq_ass_quartile = raccoglitore_dimensioni[dimensione][quartile]["freq_ass_quartile"]
            freq_ass_all = raccoglitore_dimensioni[dimensione][quartile]["freq_ass_all"]
            freq_ass_sovra = raccoglitore_dimensioni[dimensione][quartile]["freq_ass_sovra"]
            
            # calcolo i p-value
            
            p_value_quartile_all = group_comparison([freq_ass_quartile,freq_ass_all])
            p_value_sovra_all = group_comparison([freq_ass_sovra,freq_ass_all])
            
            
            
                
            
            # per calculare il p-value del quartile rispetto alla sovra categoria devo eliminare i valori 0 => poichè ci possono essere valori che non ci sono ne nel quartile ne nella sovra categoria
            
            for i in range(0,len(freq_ass_sovra)):
                if freq_ass_sovra[i] == 0 or freq_ass_sovra[i] == "0":
                    freq_ass_sovra.pop(i)
                    freq_ass_quartile.pop(i)
                
            p_value_quartile_sovra = group_comparison([freq_ass_quartile,freq_ass_sovra])
            
            '''
            
            groups = []
            
            for row in raccoglitore_dimensioni[dimensione][quartile]:
                group = [raccoglitore_dimensioni[dimensione][quartile]["freq_ass_quartile"],raccoglitore_dimensioni[dimensione][quartile]["freq_ass_sovra"]]
                groups.append(group)
                
            p_value_quartile_sovra = group_comparison(groups)
            
            groups = []
            
            for row in raccoglitore_dimensioni[dimensione][quartile]:
                group = [raccoglitore_dimensioni[dimensione][quartile]["freq_ass_quartile"],raccoglitore_dimensioni[dimensione][quartile]["freq_ass_all"]]
                groups.append(group)
                
            p_value_quartile_all = group_comparison(groups)
            
            
            groups = []
            
            for row in raccoglitore_dimensioni[dimensione][quartile]:
                group = [raccoglitore_dimensioni[dimensione][quartile]["freq_ass_sovra"],raccoglitore_dimensioni[dimensione][quartile]["freq_ass_all"]]
                groups.append(group)
                
            p_value_sovra_all = group_comparison(groups)
            
            print(p_value_quartile_sovra,p_value_quartile_all,p_value_sovra_all)
            '''
            
            # calcolo p-trend
            if dimensione in ignora_per_ptrend:
                p_trend = "NULL"
            else:
                
                if usa_mapping == False:
                
                    x = raccoglitore_dimensioni[dimensione][quartile]["valore"].astype(float)
                    y = [i for i in raccoglitore_dimensioni[dimensione][quartile][["freq_ass_quartile"]].astype(float)["freq_ass_quartile"]]
                
                    p_trend = linregress(x,y)[3]
                    
                else:
                    x = []
                    for valore in raccoglitore_dimensioni[dimensione][quartile]["valore"]:
                        x.append(mapping_per_ptrend[dimensione][valore])
                    y = [i for i in raccoglitore_dimensioni[dimensione][quartile][["freq_ass_quartile"]].astype(float)["freq_ass_quartile"]]
                
                    p_trend = linregress(x,y)[3]
            
            
            # creo row
            
            row = [[p_value_quartile_all,
                p_value_quartile_sovra,
                p_value_sovra_all,
                p_trend
            ]]
            
            # creo il df
            
            raccoglitore_pvalue[dimensione][quartile] = pd.DataFrame(row, columns=["pvalue_quartile_all", "pvalue_quartile_sovra", "pvalue_sovra_all", "p_trend"])
    
    return raccoglitore_pvalue

In [ ]:
from matplotlib.colors import to_rgba

def crea_barplot_all_anova(raccoglitore_dimensioni,tabella_p_value,dimensione,valore,rotation = None,inverted_axes = False):

    fig, ax = plt.subplots(nrows=2, ncols=2, figsize=(15,10))

    if valore == "label":
        fig.tight_layout(h_pad=10, w_pad=5)
    else:
        fig.tight_layout(pad=8.5)
    
    for quartile in quartili:
        
        sub = raccoglitore_dimensioni[dimensione][quartile].sort_values("valore", ascending=False)
        sub_freq_rel = []
        for index, row in sub.iterrows():
            #for col in ["freq_rel_quartile", "freq_rel_sovra", "freq_rel_all"]:
            for col in ["freq_rel_quartile", "freq_rel_sovra"]:
            
                #new_row = [row["valore"],col, sub.at[index,col]]
                new_row = [row[valore],col, sub.at[index,col]]
                sub_freq_rel.append(new_row)

        sub_freq_rel = pd.DataFrame(sub_freq_rel, columns=["valore", "colonna", "frequenza"])
        sub_freq_rel = sub_freq_rel.sort_values(by=["valore"], ascending=True)
        
        if quartile == "wrong_false":
            i = 0
            j = 0
        elif quartile == "wrong_true":
            i = 0
            j = 1
        elif quartile == "correct_false":
            i = 1
            j = 0
        elif quartile == "correct_true":
            i = 1
            j = 1
        
        
        ax[i,j].grid(alpha=0.4)
            
        # colori
        color_dict = {'freq_rel_quartile': to_rgba('salmon', 1),
                    'freq_rel_sovra': to_rgba('paleturquoise', 0.2)
        }
                    #'freq_rel_all': to_rgba('mistyrose', 0.2)}   

        if inverted_axes:
            # orrizzontale  
            sns.barplot(data=sub_freq_rel, y = "valore", x = "frequenza", hue ="colonna", ax=ax[i,j], palette=color_dict, orient="h")   
        else:
            #verticale
            #sns.barplot(data=sub_freq_rel, y = "frequenza", x = "valore", hue ="colonna", ax=ax[i,j], palette=["g","indianred", "goldenrod"]) 
            sns.barplot(data=sub_freq_rel, y = "frequenza", x = "valore", hue ="colonna", ax=ax[i,j], palette=color_dict)
            
        
        '''
        aplha_value = 0.35
        
        for bar, alpha in zip(ax[i,j].containers[2], list(aplha_value*np.ones(len(sub_freq_rel["valore"])))):
            bar.set_alpha(alpha)
        for bar, alpha in zip(ax[i,j].containers[0], list(aplha_value*np.ones(len(sub_freq_rel["valore"])))):
            bar.set_alpha(alpha)
        '''
        
        if rotation is not None:
            ax[i,j].tick_params(axis='x', rotation=rotation)
        else:
            ax[i,j].tick_params(axis='x', rotation=30)
        
        if tabella_p_value[dimensione][quartile]["pvalue_quartile_sovra"][0] < 0.05:
            if tabella_p_value[dimensione][quartile]["p_trend"][0] != "NULL":
                ax[i,j].text(0.5, 1.08, "p-value: " + str(round(tabella_p_value[dimensione][quartile]["pvalue_quartile_sovra"][0],3)) + " -- p-trend:" + str(round(tabella_p_value[dimensione][quartile]["p_trend"][0],3))
                        , transform=ax[i,j].transAxes, fontsize=10, fontweight=0, color='red', va='top', ha='center')
                ax[i,j].set_title(dimensione + " -- " + quartile,loc="center", fontsize=15, fontweight=0, color='red', y=1.09)
            else:
                ax[i,j].text(0.5, 1.08, "p-value: " + str(round(tabella_p_value[dimensione][quartile]["pvalue_quartile_sovra"][0],3)) + " -- p-trend:" + tabella_p_value[dimensione][quartile]["p_trend"][0]
                        , transform=ax[i,j].transAxes, fontsize=10, fontweight=0, color='red', va='top', ha='center')
                ax[i,j].set_title(dimensione + " -- " + quartile,loc="center", fontsize=15, fontweight=0, color='red', y=1.09)
            
        else:
            if tabella_p_value[dimensione][quartile]["p_trend"][0] != "NULL":
                ax[i,j].text(0.5, 1.08, "p-value: " + str(round(tabella_p_value[dimensione][quartile]["pvalue_quartile_sovra"][0],3)) + " -- p-trend:" + str(round(tabella_p_value[dimensione][quartile]["p_trend"][0],3))
                        , transform=ax[i,j].transAxes, fontsize=10, fontweight=0, color='black', va='top', ha='center')
                ax[i,j].set_title(dimensione + " -- " + quartile,loc="center", fontsize=15, fontweight=0, color='black', y=1.09)
            else:
                ax[i,j].text(0.5, 1.08, "p-value: " + str(round(tabella_p_value[dimensione][quartile]["pvalue_quartile_sovra"][0],3)) + " -- p-trend:" + tabella_p_value[dimensione][quartile]["p_trend"][0]
                        , transform=ax[i,j].transAxes, fontsize=10, fontweight=0, color='black', va='top', ha='center')
                ax[i,j].set_title(dimensione + " -- " + quartile,loc="center", fontsize=15, fontweight=0, color='black', y=1.09)
            
        
    
    # sistemo legende
    for i in range(2):
        for j in range(2):
            if i == 1 and j == 1:
                ax[i,j].legend(bbox_to_anchor=(1.1, 0.35), loc='upper left', borderaxespad=0)
            else:
                ax[i,j].get_legend().remove()
        
    
   
    plt.show()

In [ ]:
from statsmodels.formula.api import ols
import statsmodels.api as sm
from statsmodels.stats.multicomp import pairwise_tukeyhsd

def anova_table(aov):
    aov['mean_sq'] = aov[:]['sum_sq']/aov[:]['df']
    aov['eta_sq'] = aov[:-1]['sum_sq']/sum(aov['sum_sq']) 
    aov['omega_sq'] = (aov[:-1]['sum_sq']-(aov[:-1]['df']*aov['mean_sq'][-1]))/(sum(aov['sum_sq'])+aov['mean_sq'][-1])
    cols = ['sum_sq', 'df', 'mean_sq', 'F', 'PR(>F)', 'eta_sq', 'omega_sq']
    aov = aov[cols]
    return aov

In [1]:
def calcola_test_mann_modificato(df, col, quartili,gruppo_confronto = ["sovra","correctness","all"],solo_significativi = False):
    df_mann = []
    for quartile in quartili:
        
        for confronto in gruppo_confronto:
        
            if confronto == "sovra":
            
                # valori del quartile
                count = df[df["quartile"] == quartile][col]
                
                # nome del quartile
                group1 = df[df["quartile"] == quartile]["quartile"].unique()
                    
                if quartile == "correct_true" or quartile == "wrong_true":
                    sovra = "true"
                else:
                    sovra = "pants-on-fire"
                
                # nomi quartili sovrapposti
                group2 = df[(df["sovra"] == sovra)]["quartile"].unique()
                
                # valori quartili sovrapposti
                nobs = df[(df["sovra"] == sovra)][col]
            
            elif confronto == "correctness":
                
                # valori del quartile
                count = df[df["quartile"] == quartile][col]
                
                # nome del quartile
                group1 = df[df["quartile"] == quartile]["quartile"].unique()
                    
                if quartile == "correct_true" or quartile == "correct_false":
                    # nomi quartili sovrapposti
                    group2 = df[(df["quartile"] == "correct_true") | (df["quartile"] == "correct_false")]["quartile"].unique()
                    
                    # valori quartili sovrapposti
                    nobs = df[(df["quartile"] == "correct_true") | (df["quartile"] == "correct_false")][col]
                else:
                    # nomi quartili sovrapposti
                    group2 = df[(df["quartile"] == "wrong_true") | (df["quartile"] == "wrong_false")]["quartile"].unique()
                    
                    # valori quartili sovrapposti
                    nobs = df[(df["quartile"] == "wrong_true") | (df["quartile"] == "wrong_false")][col]
                
                
            elif confronto == "all":
                
                # valori del quartile
                count = df[df["quartile"] == quartile][col]
                
                # nome del quartile
                group1 = df[df["quartile"] == quartile]["quartile"].unique()
                    
                group2 = "[all]"
                    
                # valori quartili sovrapposti
                nobs = df[col]
            
            test = test_mann_whitney_u(count,nobs)
                
            if solo_significativi:
                if test < 0.05:
                    print("-" * 100)
                    print("Test Mann: " + quartile + " vs " + confronto+ " -> dimensione = "+ col)
                    
                    print(group1,group2,"--> pvalue:", round(test,3))
                    print([count.mean()],[nobs.mean()])
                    df_mann.append([col,quartile,confronto,round(test,3)])
            else:
                print("-" * 100)
                print("Test Mann: " + quartile + " vs " + confronto + " -> dimensione = "+ col)
                print(group1,group2,"--> pvalue: ", round(test,3))
                print([count.mean()],[nobs.mean()])
                
    return pd.DataFrame(df_mann, columns = ["dimensione","quartile","gruppo_confronto","pvalue"])

In [2]:
def crea_boxplot_e_stripplot_modificato(df, quartile, y_dimensione,to_highligth):
    if quartile == "wrong_false":
        df = df.copy()
        
        df_quartile = df[df["quartile"] == "wrong_false"]
        df_quartile["gruppo"] = "wrong_false"
        
        df_analisi = df_quartile.copy()
        
        if "sovra" in to_highligth:
            df = df.copy()
            df_sovra = df[df["sovra"] == "pants-on-fire"]
            df_sovra["gruppo"] = "sovra"
            df_analisi = pd.concat([df_analisi,df_sovra])
        
        if "correctness" in to_highligth:
            df_wrong = df[(df["quartile"] == "wrong_false" )| (df["quartile"] == "wrong_true")]
            df_wrong["gruppo"] = "correctness"
            df_analisi = pd.concat([df_analisi,df_wrong])
            
        if "all" in to_highligth:
            df_all_group = df.copy()
            df_all_group["gruppo"] = "all"
            df_analisi = pd.concat([df_analisi,df_all_group])
    
    elif quartile == "wrong_true":
        df = df.copy()
        
        df_quartile = df[df["quartile"] == "wrong_true"]
        df_quartile["gruppo"] = "wrong_true"
        
        df_analisi = df_quartile.copy()
        
        if "sovra" in to_highligth:
            df = df.copy()
            df_sovra = df[df["sovra"] == "true"]
            df_sovra["gruppo"] = "sovra"
            df_analisi = pd.concat([df_analisi,df_sovra])
        
        if "correctness" in to_highligth:
            df_wrong = df[(df["quartile"] == "wrong_false" )| (df["quartile"] == "wrong_true")]
            df_wrong["gruppo"] = "correctness"
            df_analisi = pd.concat([df_analisi,df_wrong])
            
        if "all" in to_highligth:
            df_all_group = df.copy()
            df_all_group["gruppo"] = "all"
            df_analisi = pd.concat([df_analisi,df_all_group])
    
    elif quartile == "correct_false":
        df = df.copy()
        
        df_quartile = df[df["quartile"] == "correct_false"]
        df_quartile["gruppo"] = "correct_false"
        
        df_analisi = df_quartile.copy()
        
        if "sovra" in to_highligth:
            df = df.copy()
            df_sovra = df[df["sovra"] == "pants-on-fire"]
            df_sovra["gruppo"] = "sovra"
            df_analisi = pd.concat([df_analisi,df_sovra])
        
        if "correctness" in to_highligth:
            df_wrong = df[(df["quartile"] == "correct_false" )| (df["quartile"] == "correct_true")]
            df_wrong["gruppo"] = "correctness"
            df_analisi = pd.concat([df_analisi,df_wrong])
            
        if "all" in to_highligth:
            df_all_group = df.copy()
            df_all_group["gruppo"] = "all"
            df_analisi = pd.concat([df_analisi,df_all_group])
    
    elif quartile == "correct_true":
        df = df.copy()
        
        df_quartile = df[df["quartile"] == "correct_true"]
        df_quartile["gruppo"] = "correct_true"
        
        df_analisi = df_quartile.copy()
        
        if "sovra" in to_highligth:
            df = df.copy()
            df_sovra = df[df["sovra"] == "true"]
            df_sovra["gruppo"] = "sovra"
            df_analisi = pd.concat([df_analisi,df_sovra])
        
        if "correctness" in to_highligth:
            df_wrong = df[(df["quartile"] == "correct_true" )| (df["quartile"] == "correct_false")]
            df_wrong["gruppo"] = "correctness"
            df_analisi = pd.concat([df_analisi,df_wrong])
            
        if "all" in to_highligth:
            df_all_group = df.copy()
            df_all_group["gruppo"] = "all"
            df_analisi = pd.concat([df_analisi,df_all_group])
    else:
        print("errore quartile")
        return
    

    ax = sns.boxplot(palette = "pastel",x="gruppo", y=y_dimensione, data=df_analisi)
    '''
    i = 0
    for patch in ax.patches:
        if ("sovra" not in to_highligth and i == 1) or ("correctness" not in to_highligth and i == 2) or ("all" not in to_highligth and i == 3):
            r, g, b, a = patch.get_facecolor()
            patch.set_facecolor((r, g, b, .05))
            patch.remove()
        i += 1
    '''
    ax = sns.stripplot(palette = "dark",x="gruppo", y=y_dimensione, data=df_analisi, jitter=0.2, size=5, alpha=0.95)
    
    
    # add title
    plt.title("analisi " + y_dimensione , loc="center", fontsize=15, fontweight=0, color='black', y=1.01)
    plt.grid(alpha=0.3)
    #ax.tick_params(axis='x', rotation=90)
    # show the graph
    #plt.show()
    return ax